
# Pytorch + HuggingFace 
# KcElectra Model
**KcELECTRA**-Base FineTune with PyTorch-Lightning1.3 사용 <br>
https://www.kaggle.com/code/junbumlee/kcelectra-base-finetune-with-pytorch-lightning1-3/notebook

## Datasets
- 지역 20곳 & 각 지역별 식당 40여개 리뷰 전체 크롤링 <br>
- 강릉, 건대, 경주, 광안리, 남포동, 부평, 사당, 서귀포, 서면, 속초, 송도, 연신내, 왕십리, 일산, 전주한옥마을, 제주, 판교, 해운대, 혜화, 홍대

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
- GPU로 돌려주세요

#Preparation

In [1]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 설치
!pip install transformers # HuggingFace transformers
!pip install pandas tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 35.6 MB/s 
     |████████████████████████████████| 7.6 MB 61.2 MB/s 
     |████████████████████████████████| 182 kB 81.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW, TextClassificationPipeline
from tqdm.notebook import tqdm

In [4]:
# 데이터셋 불러오기
train = pd.read_csv('/content/drive/MyDrive/train_set (1).csv')
test = pd.read_csv('/content/drive/MyDrive/test_set (1).csv')
test_pd = pd.read_csv('/content/drive/MyDrive/test_set (1).csv')

In [5]:
train=train.rename(columns={'평점':'score'})

In [6]:
train["라벨링"] = train.score.map({0.5:0,1:1,1.5:2,2:3,2.5:4,3:5,3.5:6,4:7,4.5:8,5:9})
train

,리뷰,score,라벨링
0,국물이 냄새도없고 담백하고 맛있었어요,3.5,6.0
1,뷰 맛집~ \n근데 파스타는 별로에요.,3.0,5.0
2,옛날맛에서 변했어요 감튀도 냉동싸구려고 햄버거 내용물이 많이 짜졌어요,1.5,2.0
3,생각했던 스페인요리와는 조금 다르네요. 그래도 직원분들이 친절하십니다.,3.0,5.0
4,마이크로 웨이팅 번호 부르고 어디 앉아라 계속 말하는통에 음식이 입으로 들어가는지 ...,2.0,3.0
...,...,...,...
30249,배터지게 먹고 갑니다!,5.0,9.0
30250,항상 너무 친절하고 맛있는 다이닝룸이에요!!,5.0,9.0
30251,친절하고 맛있었어요,4.0,7.0
30252,맛은있는데 가격대비 양이 너믄 작아요,4.0,7.0


In [7]:
test=test.rename(columns={'평점':'grade'})

In [8]:
test["라벨링"] = test.grade.map({0.5:0,1:1,1.5:2,2:3,2.5:4,3:5,3.5:6,4:7,4.5:8,5:9})
test

,리뷰,grade,라벨링
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,9
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,9
2,좋아요,5.0,9
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,9
4,귀엽다,3.0,5
...,...,...,...
27924,생각보다 그냥 그래욤,3.5,6
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,9
27926,친절해요,5.0,9
27927,좋아요,5.0,9


In [9]:
test = test.applymap(str)

In [10]:
train = train.applymap(str)

In [11]:
train.to_csv('train_set.csv')

In [12]:
test.to_csv('test_set.csv')

In [13]:
train= './train_set.csv' 
test = './test_set.csv'

In [14]:
# GPU 사용
device = torch.device("cuda")

# Dataset

In [15]:
class ClassifyDataset(Dataset):
  
  def __init__(self, csv_file):
    # csv 읽어오기 & NaN 처리
    self.dataset = pd.read_csv(csv_file).dropna(axis=0) 
    # 중복 제거
    self.dataset.drop_duplicates(subset=['리뷰'], inplace=True)
    # pretrained model
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base") 

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:4].values # idx 행과 0,1,2 columns
    text = row[1]
    y = row[3]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt', # return pytorch tensors
        truncation=True, # reducing long sequences
        max_length=256,
        pad_to_max_length=True, # padding
        add_special_tokens=True # 문장 앞뒤로 special tocken - padding 부착
        )
    
    input_ids = inputs['input_ids'][0] 
    attention_mask = inputs['attention_mask'][0] # the position of the padded indices so that the model does not attend to them

    return input_ids, attention_mask, y

In [16]:
train_dataset = ClassifyDataset(train)
test_dataset = ClassifyDataset(test)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

         Unnamed: 0         score           라벨링
count  24689.000000  24689.000000  24689.000000
mean   14981.643525      3.717425      6.434850
std     8770.486945      1.400143      2.800286
min        0.000000      0.500000      0.000000
25%     7327.000000      3.000000      5.000000
50%    14987.000000      4.000000      7.000000
75%    22510.000000      5.000000      9.000000
max    30253.000000      5.000000      9.000000
         Unnamed: 0         grade           라벨링
count  22951.000000  22951.000000  22951.000000
mean   13748.983138      4.484619      7.969239
std     8091.051841      0.939953      1.879905
min        0.000000      0.500000      0.000000
25%     6704.000000      4.000000      7.000000
50%    13635.000000      5.000000      9.000000
75%    20740.500000      5.000000      9.000000
max    27928.000000      5.000000      9.000000


# Create Model & Train

In [17]:
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base",num_labels=10).to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model,device=0) # gpu로 변환

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

In [18]:
epochs = 10
batch_size = 32

optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
losses = []
accuracies = []

for i in range(epochs): # epoch 5
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train() # forward

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader): # tqdm 진행상황 확인 
    optimizer.zero_grad()
  
    y_batch = y_batch.type(torch.LongTensor)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0] 
    loss = F.cross_entropy(y_pred, y_batch)

    loss.backward()
    optimizer.step() 

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total) 

  0%|          | 0/772 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 213.34736502170563 Accuracy: tensor(0.3728, device='cuda:0')
Batch Loss: 401.8804997205734 Accuracy: tensor(0.3831, device='cuda:0')
Batch Loss: 572.8336840867996 Accuracy: tensor(0.4073, device='cuda:0')
Batch Loss: 735.1682580709457 Accuracy: tensor(0.4266, device='cuda:0')
Batch Loss: 888.3323819637299 Accuracy: tensor(0.4419, device='cuda:0')
Batch Loss: 1037.197809934616 Accuracy: tensor(0.4518, device='cuda:0')
Batch Loss: 1178.2638419270515 Accuracy: tensor(0.4618, device='cuda:0')
Train Loss: 1280.0419407486916 Accuracy: tensor(0.4649, device='cuda:0')


  0%|          | 0/772 [00:00<?, ?it/s]

Batch Loss: 140.02202433347702 Accuracy: tensor(0.5281, device='cuda:0')
Batch Loss: 271.56442296504974 Accuracy: tensor(0.5425, device='cuda:0')
Batch Loss: 403.9022355079651 Accuracy: tensor(0.5447, device='cuda:0')
Batch Loss: 536.8941769599915 Accuracy: tensor(0.5450, device='cuda:0')
Batch Loss: 665.2069050073624 Accuracy: tensor(0.5488, device='cuda:0')
Batch Loss: 792.8689961433411 Accuracy: tensor(0.5495, device='cuda:0')
Batch Loss: 921.7827845811844 Accuracy: tensor(0.5499, device='cuda:0')
Train Loss: 1012.1722192764282 Accuracy: tensor(0.5513, device='cuda:0')


...

Batch Loss: 103.35911583900452 Accuracy: tensor(0.6162, device='cuda:0')
Batch Loss: 207.87712180614471 Accuracy: tensor(0.6156, device='cuda:0')
Batch Loss: 310.3884808421135 Accuracy: tensor(0.6197, device='cuda:0')
Batch Loss: 413.51391357183456 Accuracy: tensor(0.6202, device='cuda:0')
Batch Loss: 516.4275148510933 Accuracy: tensor(0.6224, device='cuda:0')
Batch Loss: 619.3514006733894 Accuracy: tensor(0.6236, device='cuda:0')
Batch Loss: 724.8178217411041 Accuracy: tensor(0.6217, device='cuda:0')
Train Loss: 799.5263996720314 Accuracy: tensor(0.6215, device='cuda:0')


  0%|          | 0/772 [00:00<?, ?it/s]

Batch Loss: 97.89503049850464 Accuracy: tensor(0.6400, device='cuda:0')
Batch Loss: 195.8644660115242 Accuracy: tensor(0.6387, device='cuda:0')
Batch Loss: 294.21361953020096 Accuracy: tensor(0.6402, device='cuda:0')
Batch Loss: 393.7230234146118 Accuracy: tensor(0.6387, device='cuda:0')
Batch Loss: 493.22663563489914 Accuracy: tensor(0.6365, device='cuda:0')
Batch Loss: 592.5259029865265 Accuracy: tensor(0.6355, device='cuda:0')
Batch Loss: 695.4177329540253 Accuracy: tensor(0.6338, device='cuda:0')
Train Loss: 766.6996917426586 Accuracy: tensor(0.6349, device='cuda:0')


  0%|          | 0/772 [00:00<?, ?it/s]

Batch Loss: 94.99323612451553 Accuracy: tensor(0.6472, device='cuda:0')
Batch Loss: 188.0543667972088 Accuracy: tensor(0.6547, device='cuda:0')
Batch Loss: 286.76347157359123 Accuracy: tensor(0.6478, device='cuda:0')
Batch Loss: 381.3389356434345 Accuracy: tensor(0.6488, device='cuda:0')
Batch Loss: 477.1209537088871 Accuracy: tensor(0.6483, device='cuda:0')
Batch Loss: 573.8120876848698 Accuracy: tensor(0.6477, device='cuda:0')
Batch Loss: 669.223854392767 Accuracy: tensor(0.6486, device='cuda:0')
Train Loss: 739.3022281229496 Accuracy: tensor(0.6472, device='cuda:0')


In [20]:
losses, accuracies

([1280.0419407486916,
  1012.1722192764282,
  947.9222384691238,
  908.9944158196449,
  881.7673169970512,
  855.388806283474,
  822.7128472924232,
  799.5263996720314,
  766.6996917426586,
  739.3022281229496],
 [tensor(0.4649, device='cuda:0'),
  tensor(0.5513, device='cuda:0'),
  tensor(0.5649, device='cuda:0'),
  tensor(0.5753, device='cuda:0'),
  tensor(0.5855, device='cuda:0'),
  tensor(0.5965, device='cuda:0'),
  tensor(0.6114, device='cuda:0'),
  tensor(0.6215, device='cuda:0'),
  tensor(0.6349, device='cuda:0'),
  tensor(0.6472, device='cuda:0')])

In [21]:
# 모델 저장하기
torch.save(model.state_dict(), "naver_map_model.pt") 

In [22]:
# 모델 불러오기
model.load_state_dict(torch.load("./naver_map_model.pt"))

<All keys matched successfully>

# TEST

## Evaluation

In [23]:
# 테스트 데이터셋 정확도 확인하기

model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.type(torch.LongTensor)
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/1435 [00:00<?, ?it/s]

Accuracy: tensor(0.6507, device='cuda:0')


In [24]:
# 리뷰 텍스트 predicted label(0,1) 확인

def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=256
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# gradient 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
        
    logits = outputs[0] 
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits) 
    return result

In [25]:
sentences_predict('맛없음')

0

In [26]:
sentences_predict('생각보다 맛있고, 사장님이 친절해요')

9

## Test Result

In [27]:
score = [] # label - score

total_len = len(test_pd)

for cnt, review in enumerate(test_pd['리뷰']): 
  pred = sentiment_classifier(review) 
  score.append(pred) 
  print(cnt, '개 문장 분류 완료')

0 개 문장 분류 완료
1 개 문장 분류 완료
2 개 문장 분류 완료
3 개 문장 분류 완료
4 개 문장 분류 완료
5 개 문장 분류 완료
6 개 문장 분류 완료
7 개 문장 분류 완료
8 개 문장 분류 완료
9 개 문장 분류 완료
10 개 문장 분류 완료
11 개 문장 분류 완료
12 개 문장 분류 완료
13 개 문장 분류 완료
14 개 문장 분류 완료
15 개 문장 분류 완료
16 개 문장 분류 완료
17 개 문장 분류 완료
18 개 문장 분류 완료
19 개 문장 분류 완료
20 개 문장 분류 완료


/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
22929 개 문장 분류 완료
22930 개 문장 분류 완료
22931 개 문장 분류 완료
22932 개 문장 분류 완료
22933 개 문장 분류 완료
22934 개 문장 분류 완료
22935 개 문장 분류 완료
22936 개 문장 분류 완료
22937 개 문장 분류 완료
22938 개 문장 분류 완료
22939 개 문장 분류 완료
22940 개 문장 분류 완료
22941 개 문장 분류 완료
22942 개 문장 분류 완료
22943 개 문장 분류 완료
22944 개 문장 분류 완료
22945 개 문장 분류 완료
22946 개 문장 분류 완료
22947 개 문장 분류 완료
22948 개 문장 분류 완료
22949 개 문장 분류 완료
22950 개 문장 분류 완료
22951 개 문장 분류 완료
22952 개 문장 분류 완료
22953 개 문장 분류 완료
22954 개 문장 분류 완료
22955 개 문장 분류 완료
22956 개 문장 분류 완료
22957 개 문장 분류 완료
22958 개 문장 분류 완료
22959 개 문장 분류 완료
22960 개 문장 분류 완료
22961 개 문장 분류 완료
22962 개 문장 분류 완료
22963 개 문장 분류 완료
22964 개 문장 분류 완료
22965 개 문장 분류 완료
22966 개 문장 분류 완료
22967 개 문장 분류 완료
22968 개 문장 분류 완료
22969 개 문장 분류 완료
22970 개 문장 분류 완료
22971 개 문장 분류 완료
22972 개 문장 분류 완료
22973 개 문장 분류 완료
22974 개 문장 분류 완료
22975 개 문장 분류 완료
22976 개 문장 분류 완료
22977 개 문장 분류 완료
22978 개 문장 분류 완료
22979 개 문장 분류 완료
22980 개 문장 분류 완료
22981 개 문장 분류 완료
22982 개 문장 분류 완료
22983 개 문장 분류 완료
22984 개 문장 분류 완료
22985 개 문장 분

In [28]:
score

[[{'label': 'LABEL_9', 'score': 0.8713719248771667}],
 [{'label': 'LABEL_9', 'score': 0.9576641917228699}],
 [{'label': 'LABEL_5', 'score': 0.533068835735321}],
 [{'label': 'LABEL_9', 'score': 0.9607484340667725}],
 [{'label': 'LABEL_6', 'score': 0.4036368429660797}],
 [{'label': 'LABEL_0', 'score': 0.5124117732048035}],
 [{'label': 'LABEL_5', 'score': 0.5423186421394348}],
 [{'label': 'LABEL_9', 'score': 0.7405596375465393}],
 [{'label': 'LABEL_9', 'score': 0.5529360175132751}],
 [{'label': 'LABEL_9', 'score': 0.9502641558647156}],
 [{'label': 'LABEL_9', 'score': 0.9252052307128906}],
 [{'label': 'LABEL_9', 'score': 0.9198362827301025}],
 [{'label': 'LABEL_9', 'score': 0.4356224834918976}],
 [{'label': 'LABEL_9', 'score': 0.9617881178855896}],
 [{'label': 'LABEL_9', 'score': 0.8421443104743958}],
 [{'label': 'LABEL_9', 'score': 0.9799679517745972}],
 [{'label': 'LABEL_9', 'score': 0.9466381072998047}],
 [{'label': 'LABEL_9', 'score': 0.9315677285194397}],
 [{'label': 'LABEL_9', 'score

In [29]:
test_pd['predicted'] = 0
score_list = [] # score 

for i in range(len(score)): 
    test_pd['predicted'][i] = int(score[i][0].get('label')[-1])
    score_list.append(float(score[i][0].get('score')))

<ipython-input-29-8d08ee1f72d3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pd['predicted'][i] = int(score[i][0].get('label')[-1])


In [30]:
test_pd

,리뷰,평점,라벨링,predicted
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,1,9
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,1,9
2,좋아요,5.0,1,5
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,1,9
4,귀엽다,3.0,0,6
...,...,...,...,...
27924,생각보다 그냥 그래욤,3.5,0,5
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,1,9
27926,친절해요,5.0,1,6
27927,좋아요,5.0,1,5


In [31]:
score_df = pd.DataFrame({'score':score_list})
final_test_pd = pd.concat([test_pd,score_df],axis=1)
final_test_pd

,리뷰,평점,라벨링,predicted,score
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,1,9,0.871372
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,1,9,0.957664
2,좋아요,5.0,1,5,0.533069
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,1,9,0.960748
4,귀엽다,3.0,0,6,0.403637
...,...,...,...,...,...
27924,생각보다 그냥 그래욤,3.5,0,5,0.671368
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,1,9,0.340487
27926,친절해요,5.0,1,6,0.422172
27927,좋아요,5.0,1,5,0.533069


In [32]:
# 결과 저장
final_test_pd.to_csv("./final_result.csv", encoding = 'utf-8-sig') 

In [33]:
# 전체 결과 정확도 

from sklearn.metrics import classification_report
print(classification_report(final_test_pd['predicted'], final_test_pd['라벨링'])) 

              precision    recall  f1-score   support

           0       0.15      0.93      0.26       572
           1       0.00      0.04      0.00       187
           3       0.00      0.00      0.00       383
           5       0.00      0.00      0.00      4972
           6       0.00      0.00      0.00      2037
           7       0.00      0.00      0.00      1864
           8       0.00      0.00      0.00        40
           9       0.00      0.00      0.00     17874

    accuracy                           0.02     27929
   macro avg       0.02      0.12      0.03     27929
weighted avg       0.00      0.02      0.01     27929



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
